# AirBNB tracker

In [1]:
import numpy as np
import datetime
import pandas as pd
import pyperclip
import pickle
import re

In [2]:
tous_les_logements = pickle.load(open('tous_les_logements.p', 'rb'))
print(tous_les_logements.shape)
tous_les_logements.tail(2)

(1979, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
1977,2024-03-27,Hébergement ⋅ Bagneux\nKXX Complexe privé 15mi...,Hébergement,Bagneux,KXX Complexe privé 15min PARIS ParcExpo 12 cha...,4570,NaN,NaN,NaN,NaN,False,False
1978,2024-03-27,Appartement ⋅ Villejuif\nMagnifique T2 a 10 mi...,Appartement,Villejuif,Magnifique T2 a 10 minute de Paris,505,NaN,NaN,NaN,NaN,False,False


In [3]:
class Logement:
    __slots__ = [
        'date_enregistrement',
        'texte',
        'type',
        'ville',
        'description',
        'prix',
        'note',
        'nb_avis',
        'nb_lits',
        'nb_chambres',
        'est_professionnel',
        'est_nouveau'
    ]
    def __init__(self, ville: str, type: str, description: str,
                 nb_lits: int, nb_chambres: int,
                 est_professionnel: bool, est_nouveau: bool,
                 prix: int, note: float, nb_avis: int,
                 texte: str):
        self.type = type
        self.ville = ville
        self.description = description
        self.nb_lits = nb_lits
        self.nb_chambres = nb_chambres
        self.prix = prix
        self.est_nouveau = est_nouveau
        self.est_professionnel = est_professionnel
        self.note = note
        self.nb_avis = nb_avis

        self.date_enregistrement = datetime.date.today()
        self.texte = texte
    
    @classmethod
    def from_text(cls, texte):
        texte = texte.strip()
        attrs = set(cls.__slots__) - {'texte', 'date_enregistrement'}
        kwargs = dict(texte=texte)
        for attr in attrs:
            kwargs[attr] = eval(f'cls._get_{attr}')(texte)
        return cls(**kwargs)
    
    
    @staticmethod
    def _get_type(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[0]
        #return texte.split('\n')[0].split(pat)[0]
    
    @staticmethod
    def _get_ville(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[2]
        #return texte.split('\n')[0].split(' ⋅ ')[1]

    @staticmethod
    def _get_description(text: str):
        return text.split('\n')[1]
    
    @staticmethod
    def _get_nb_lits(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[0]

    @staticmethod
    def _get_nb_chambres(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[1]
    
    @staticmethod
    def _get_nb_lits_nb_chambres(texte: str) -> tuple:
        nb_lits, nb_chambres = None, None
        pat = r'(?P<nb_lits>\d) lits?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_lits = int(rgx.group()[0])
        pat = r'(?P<nb_chambres>\d) chambres?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_chambres = int(rgx.group()[0])
        return nb_lits, nb_chambres
    
    @staticmethod
    def _get_est_professionnel(texte: str):
        return texte.split('\n')[3] == 'Professionnel'
    
    @staticmethod
    def _get_prix(texte: str):
        for ligne in texte.split('\n'):
            if rgx := re.match('(?P<prix>[\d\u202f]+) € par nuit', ligne):
                return int(rgx.groups()[0].replace('\u202f', ''))
        raise ValueError('Pas de prix trouvé sur :\n' + texte)
    
    @staticmethod
    def _get_est_nouveau(texte: str):
        return texte.split('\n')[-1] == 'Nouveau'
    
    @staticmethod
    def _get_note(texte: str):
        return Logement._get_note_et_nb_avis(texte)[0]

    @staticmethod
    def _get_nb_avis(texte: str):
        return Logement._get_note_et_nb_avis(texte)[1]

    @staticmethod
    def _get_note_et_nb_avis(texte: str):
        note, nb_avis = None, None
        pat = r'(?P<note>\d\,\d\d?) \((?P<nb_avis>\d+)\)'
        if rgx := re.search(pat, texte):
            note = float(rgx.groupdict()['note'].replace(',', '.'))
            nb_avis = int(rgx.groupdict()['nb_avis'])
        return note, nb_avis
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}('
        for attr in ('type', 'ville', 'prix'):
            rep += f'\n    {attr}={getattr(self, attr)},'
        rep = rep[:-1] + '\n)'
        return rep


def tronque_à_gauche(texte: str) -> str:
    """Tronque à gauche le texte avant 'Classement des résultats'"""
    stop = texte.find('Classement des résultats')
    return texte[stop + len('Classement des résultats'):].strip()

def tronque_à_droite(texte: str) -> str:
    """Tronque à droite le texte qui dépasse de la numérotation des pages"""
    # stop = texte.find('1\n2\n3\n4')
    stop = re.search(r'[\d…]\n[\d…]\n[\d…]', texte)
    if stop:
        return texte[:stop.start()].strip()
    stop = re.search(r'Carte Google \d+ séjours affichés.', texte)
    texte = texte[:stop.start()].strip()
    stop = re.search(r'Un voyage, deux logements', texte)
    if stop:
        texte = texte[:stop.start()].strip()
        
    pat = '\n(Appartement|Chambre|Hébergement|Tiny house|Appartement en résidence|Villa|Maison de ville) [⋅·] .+€\n'
    stop = re.search(pat, texte)
    if stop:
        texte = texte[:stop.start()].strip()
    return texte

In [55]:
page = pyperclip.paste()

page = tronque_à_gauche(page)
page = tronque_à_droite(page)

In [56]:
pat = "(Appartement|Chambre|Hébergement|Loft|Tiny house|Appartement en résidence|Villa|Maison de ville|Maison d'hôtes) [⋅·] "
splits = [
    match.start()
    for match in re.finditer(pat, page)
]
splits += [len(page)]

logements = []
for texte in [page[start:stop] for start, stop in zip(splits, splits[1:])]:
    logements.append(Logement.from_text(texte))

In [57]:
dico = [{attr: getattr(logement, attr) for attr in Logement.__slots__} for logement in logements]
page_df = pd.DataFrame(dico, index=range(len(dico)))

In [58]:
#for t in page_df.texte.unique(): print(t, '\n\n')
page_df.head(3)

,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
0,2024-04-06,Hébergement ⋅ L'Haÿ-les-Roses\nMaison familial...,Hébergement,L'Haÿ-les-Roses,Maison familiale 'Californie' aux Portes de Paris,1970,4.86,7.0,None,None,False,False
1,2024-04-06,Hébergement ⋅ Bagneux\nKXX Complexe privé 15mi...,Hébergement,Bagneux,KXX Complexe privé 15min PARIS ParcExpo 12 cha...,4570,NaN,NaN,None,None,False,False
2,2024-04-06,Loft ⋅ Bagneux\nParis à 3 min !\nArena Paris S...,Loft,Bagneux,Paris à 3 min !,799,NaN,NaN,None,None,False,False


In [59]:
assert page_df.shape[0] >= 18
assert page_df.drop(['nb_lits', 'nb_chambres'], axis=1).notnull().sum().min() > 0

AssertionError: 

In [60]:
# On ajoute à tous les logements la page en cours de scrapping : 
tous_les_logements = pd.concat([tous_les_logements, page_df], axis=0, ignore_index=True)
print(tous_les_logements.shape)
tous_les_logements.tail()

(2129, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
2124,2024-04-06,Chambre · L'Haÿ-les-Roses\nSéjournez chez Myri...,Chambre,L'Haÿ-les-Roses,Séjournez chez Myriam,796,NaN,NaN,None,NaN,False,False
2125,2024-04-06,Maison de ville ⋅ Bagneux\nMaison 10 min PARIS...,Maison de ville,Bagneux,Maison 10 min PARIS EXPO - 5 chambres/8 lits,1864,4.83,29.0,None,NaN,False,False
2126,2024-04-06,Hébergement ⋅ L'Haÿ-les-Roses\nMaison familial...,Hébergement,L'Haÿ-les-Roses,Maison familiale 'Californie' aux Portes de Paris,1970,4.86,7.0,None,None,False,False
2127,2024-04-06,Hébergement ⋅ Bagneux\nKXX Complexe privé 15mi...,Hébergement,Bagneux,KXX Complexe privé 15min PARIS ParcExpo 12 cha...,4570,NaN,NaN,None,None,False,False
2128,2024-04-06,Loft ⋅ Bagneux\nParis à 3 min !\nArena Paris S...,Loft,Bagneux,Paris à 3 min !,799,NaN,NaN,None,None,False,False


Quand on a fini d'itérer sur les pages, on dump :

In [61]:
pickle.dump(tous_les_logements.drop_duplicates().fillna(np.nan), open('tous_les_logements.p', 'wb'))